In [7]:
import awkward as ak
import numpy as np
import vector
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection
from xgboost import XGBClassifier

In [8]:
MAX_N_JETS = 6
B_TAG_THRESHOLD = 0.5

In [9]:
file_path = 'https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19980_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1_00000_0000.root'
tree_name = 'Events'
# events = NanoEventsFactory.from_root({file_path: tree_name}, schemaclass=NanoAODSchema).events()
events = NanoEventsFactory.from_root(file_path, treepath=tree_name, schemaclass=NanoAODSchema).events()

Muon = events.Muon
Electron = events.Electron
Jet = events.Jet

muon_mask = ((Muon.pt>30) & (np.abs(Muon.eta)<2.1) & (Muon.tightId) & (Muon.sip3d<4) & (Muon.pfRelIso04_all<0.15))
electron_mask = ((Electron.pt>30) & (np.abs(Electron.eta)<2.1) & (Electron.cutBased==4) & (Electron.sip3d<4))
jet_mask = ((Jet.pt>30) & (np.abs(Jet.eta)<2.4) & (Jet.isTightLeptonVeto))

muons = Muon[muon_mask]
electrons = Electron[electron_mask]
jets = Jet[jet_mask]

/Users/con_np/micromamba/envs/r3k_bdttools/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx1 => SubJet
  warnings.warn(
/Users/con_np/micromamba/envs/r3k_bdttools/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx2 => SubJet
  warnings.warn(


In [17]:
selections = PackedSelection(dtype='uint64')
# Basic selection criteria
selections.add("exactly_1l", (ak.num(electrons) + ak.num(muons)) == 1) #PackedSelection.add(name(str), selection(array), fill_value=False)
selections.add("atleast_4j", ak.num(jets) >= 4)
selections.add("exactly_1b", ak.sum(jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) == 1)
selections.add("atleast_2b", ak.sum(jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 2)
# Complex selection criteria
selections.add("4j1b", selections.all("exactly_1l", "atleast_4j", "exactly_1b")) #requirement, where all the values are True
selections.add("4j2b", selections.all("exactly_1l", "atleast_4j", "atleast_2b"))

In [18]:
region_4j2b = selections.all('4j2b')
elecs = electrons[region_4j2b]
muons = muons[region_4j2b]
jets = jets[region_4j2b]
genpart = events.GenPart[region_4j2b]

In [24]:
max(ak.num(jets, axis=1))

13

## get_permutations_dict

In [3]:
# calculate the dictionary of permutations for each number of jets
permutations_dict = {}
for n in range(4, MAX_N_JETS + 1): # for max_n_jets = 6, the range would be 4, 5, 6
    test = ak.Array(range(n)) #create array -> at least 4 jet up to 6 jets
    unzipped = ak.unzip(ak.argcartesian([test] * 4, axis=0))

    combos = ak.combinations(ak.Array(range(4)), 2, axis=0) #choosing 2 out of [0, 1, 2, 3] to create sub array (like a permutation)
    different = unzipped[combos[0]["0"]] != unzipped[combos[0]["1"]] # eg (0,1) and (0,0) would be different. combos acts like a mask. This creats boolean mask
    for i in range(1, len(combos)): #1,2,3,4,5
        different = different & (unzipped[combos[i]["0"]] != unzipped[combos[i]["1"]]) #also another boolean mask
    permutations = ak.zip([test[unzipped[i][different]] for i in range(len(unzipped))], depth_limit=1).tolist() #looks like a list of array [(),(),...,()]

    permutations = ak.concatenate([test[unzipped[i][different]][..., np.newaxis] for i in range(len(unzipped))], axis=1).to_list() #list of event of jet permutation [[],[],[],...,[]]

    permutations_dict[n] = permutations #list of permutation corresponding to 4, 5, 6 jets (24, 120, 360 permutations respectatively)

In [4]:
permutations_dict[4][19:], permutations_dict[5][115:], permutations_dict[6][355:]

([[3, 0, 2, 1], [3, 1, 0, 2], [3, 1, 2, 0], [3, 2, 0, 1], [3, 2, 1, 0]],
 [[4, 3, 0, 2], [4, 3, 1, 0], [4, 3, 1, 2], [4, 3, 2, 0], [4, 3, 2, 1]],
 [[5, 4, 2, 1], [5, 4, 2, 3], [5, 4, 3, 0], [5, 4, 3, 1], [5, 4, 3, 2]])

In [5]:
# for each permutation, calculate the corresponding label
labels_dict = {}
for n in range(4, MAX_N_JETS + 1): # 4 ,5, 6
    current_labels = []
    for inds in permutations_dict[n]: # inds are the jet permutation list
        inds = np.array(inds)
        current_label = 100 * np.ones(n)
        current_label[inds[:2]] = 24
        current_label[inds[2]] = 6
        current_label[inds[3]] = -6
        current_labels.append(current_label.tolist())

    labels_dict[n] = current_labels
    # list of length 4, 5, or 6. Elements are 24, 6, -6 or 100. (100 probably for other jets, two 24 indistinquishable, -6, 6 either from b_top or b_lep)


In [6]:
labels_dict[4][19:], labels_dict[5][115:], labels_dict[6][355:]

([[24.0, -6.0, 6.0, 24.0],
  [6.0, 24.0, -6.0, 24.0],
  [-6.0, 24.0, 6.0, 24.0],
  [6.0, -6.0, 24.0, 24.0],
  [-6.0, 6.0, 24.0, 24.0]],
 [[6.0, 100.0, -6.0, 24.0, 24.0],
  [-6.0, 6.0, 100.0, 24.0, 24.0],
  [100.0, 6.0, -6.0, 24.0, 24.0],
  [-6.0, 100.0, 6.0, 24.0, 24.0],
  [100.0, -6.0, 6.0, 24.0, 24.0]],
 [[100.0, -6.0, 6.0, 100.0, 24.0, 24.0],
  [100.0, 100.0, 6.0, -6.0, 24.0, 24.0],
  [-6.0, 100.0, 100.0, 6.0, 24.0, 24.0],
  [100.0, -6.0, 100.0, 6.0, 24.0, 24.0],
  [100.0, 100.0, -6.0, 6.0, 24.0, 24.0]])

In [7]:
# get rid of duplicates since we consider W jets to be exchangeable
# (halves the number of permutations we consider)
for n in range(4, MAX_N_JETS + 1):
    res = []
    for idx, val in enumerate(labels_dict[n]): #idx = 0, 1, 2, ... and  val = list of labels (eg. [24.0, 24.0, 6.0, -6.0] for 4 jets, [24.0, 24.0, 6.0, 100.0, -6.0] for 5 jets)
        if val in labels_dict[n][:idx]: #incremented by 1 to see if the next val is already in the previous one on lanbels_dict
            res.append(idx) #store the repated val idx
    labels_dict[n] = np.array(labels_dict[n])[res].tolist() #chosen unique permutations NOTE: val in labels[n][:idx] / val not it labels[n][:idx] give different res but the labels_dict[n] is the same (same permutation but differnet indices)
    permutations_dict[n] = np.array(permutations_dict[n])[res].tolist()

In [8]:
len(permutations_dict[4]), len(permutations_dict[5]), len(permutations_dict[6]) # after filter repeated permutations/labels 

(12, 60, 180)

In [9]:
#eval_metrices_dict
# elif include_labels and include_eval_mat:
    # these matrices tell you the overlap between the predicted label (rows) and truth label (columns). Thus, square matrix
    # the "score" in each matrix entry is the number of jets which are assigned correctly
evaluation_matrices = {}  # overall event score
for n in range(4, MAX_N_JETS + 1):
    evaluation_matrix = np.zeros((len(permutations_dict[n]), len(permutations_dict[n])))
    for i in range(len(permutations_dict[n])):
        for j in range(len(permutations_dict[n])): #going through each entries
            evaluation_matrix[i, j] = sum(np.equal(labels_dict[n][i], labels_dict[n][j])) #filling matrix

    evaluation_matrices[n] = evaluation_matrix / 4

# evaluation_matrices[4].shape = (12, 12)
# evaluation_matrices[5].shape = (60, 60)
# evaluation_matrices[6].shape = (180, 180)

# return permutations_dict, labels_dict, evaluation_matrices

## get_features

In [ ]:
def get_features(jets, electrons, muons, max_n_jets=6):
    """
    Calculate features for each of the 12 combinations per event

    Args:
        jets: selected jets
        electrons: selected electrons
        muons: selected muons
        permutations_dict: which permutations to consider for each number of jets in an event

    Returns:
        features (flattened to remove event level)
        perm_counts: how many permutations in each event. use to unflatten features
    """

In [60]:
# for four-vector addition of custom Momentum4D instances below
vector.register_awkward()

In [65]:
min(ak.num(jets)), max(ak.num(jets))

(4, 13)

In [66]:
# permutations_dict = get_permutations_dict(MAX_N_JETS) #technically from the previous function

# calculate number of jets in each event
njet = ak.num(jets).to_numpy()
# don't consider every jet for events with high jet multiplicity
njet[njet>max(permutations_dict.keys())] = max(permutations_dict.keys()) # replacing event with more than 6 jets to exactly 6 jets


In [67]:
min(njet), max(njet)

(4, 6)

In [72]:
# create awkward array of permutation indices
perms = ak.Array([permutations_dict[n] for n in njet]) # each event, depending on the jet number, will have list of permutation indices (eg. [[1, 0, 2, 3], [1, ... 1], [3, 2, 1, 0]])
perm_counts = ak.num(perms) #the number of permutations in each event (eg. [12, 60, 12, 60, 12, ... 60, 12, 12, 12])

In [90]:
#### calculate features ####
features = np.zeros((np.sum(perm_counts), 20)) #rows are the total number of pemutation ( n_event*n_jet*perm_each_jet  x  20 features )
print('events:', ak.num(jets, axis=0), '    jets:', ak.sum(ak.num(jets, axis=1)), '     perm_count:', np.sum(perm_counts))

events: 28616     jets: 135491      perm_count: 1596720


In [104]:
# grab lepton info
leptons = ak.flatten(ak.concatenate((elecs, muons), axis=1), axis=-1)

In [162]:
perms[:, 11]

<Array [[3, 2, 1, 0], [2, ... 0], [3, 2, 1, 0]] type='28616 * var * int64'>

In [168]:
perms[..., 3]

<Array [[3, 2, 3, 1, 3, 0, ... 1, 2, 0, 1, 0]] type='28616 * var * int64'>

In [183]:
### Constructing Features (filling features matrix) ###

# deltaR (b_toplep, lepton)
features[:, 0] = ak.flatten( np.sqrt((leptons.eta - jets[perms[...,3]].eta)**2 + (leptons.phi - jets[perms[...,3]].phi)**2) ).to_numpy()
# deltaR (two W)
features[:, 1] = ak.flatten( np.sqrt( (jets[perms[...,0]].eta - jets[perms[...,1]].eta)**2 + (jets[perms[...,0]].phi - jets[perms[...,1]].phi)**2 )).to_numpy()
# deltaR between W and b_tophad
features[:, 2] = ak.flatten( np.sqrt((jets[perms[...,0]].eta - jets[perms[...,2]].eta)**2 + (jets[perms[...,0]].phi - jets[perms[...,2]].phi)**2) ).to_numpy()
features[:, 3] = ak.flatten( np.sqrt((jets[perms[...,1]].eta - jets[perms[...,2]].eta)**2 + (jets[perms[...,1]].phi - jets[perms[...,2]].phi)**2) ).to_numpy()

#combining leptons and jets arrays
#creating 4momentum
el_p4 = ak.zip({'pt': elecs.pt, 'eta': elecs.eta, 'phi': elecs.phi, 'mass': elecs.mass}, with_name= 'Momentum4D')
mu_p4 = ak.zip({'pt': muons.pt, 'eta': muons.eta, 'phi': muons.phi, 'mass': muons.mass}, with_name= 'Momentum4D')
lep_p4 = ak.flatten(ak.concatenate((el_p4, mu_p4), axis=1), axis=-1)
jet_p4 = ak.zip({'pt': jets.pt, 'eta': jets.eta, 'phi': jets.phi, 'mass': jets.mass}, with_name= 'Momentum4D')

#b_toplep + lep mass
features[:, 4] = ak.flatten((jet_p4[perms[...,3]] + lep_p4).mass).to_numpy()
#Ws mass
features[:, 5] = ak.flatten((jets[perms[...,0]] + jets[perms[...,1]]).mass).to_numpy()
# W + b_tophad
features[:, 6] = ak.flatten((jets[perms[...,0]] + jets[perms[...,1]] + jets[perms[...,2]]).mass).to_numpy()
# combined pT of W and b_tophad
features[:, 7] = ak.flatten((jets[perms[...,0]] + jets[perms[...,1]] + jets[perms[...,2]]).pt).to_numpy()

# jets pt
features[:, 8] = ak.flatten(jets[perms[...,0]].pt).to_numpy()
features[:, 9] = ak.flatten(jets[perms[...,1]].pt).to_numpy()
features[:, 10] = ak.flatten(jets[perms[...,2]].pt).to_numpy()
features[:, 11] = ak.flatten(jets[perms[...,3]].pt).to_numpy()
# btagCSVV2 of every jet
features[:, 12] = ak.flatten(jets[perms[...,0]].btagCSVV2).to_numpy()
features[:, 13] = ak.flatten(jets[perms[...,1]].btagCSVV2).to_numpy()
features[:, 14] = ak.flatten(jets[perms[...,2]].btagCSVV2).to_numpy()
features[:, 15] = ak.flatten(jets[perms[...,3]].btagCSVV2).to_numpy()
# quark-gluon likelihood discriminator of every jet
features[:, 16] = ak.flatten(jets[perms[...,0]].qgl).to_numpy()
features[:, 17] = ak.flatten(jets[perms[...,1]].qgl).to_numpy()
features[:, 18] = ak.flatten(jets[perms[...,2]].qgl).to_numpy()
features[:, 19] = ak.flatten(jets[perms[...,3]].qgl).to_numpy()

# return features, perm_counts


## get_inference_result_local

In [200]:
even = (events.event%2==0)
region_selection = selections.all('4j2b')
even = even[region_selection]
even

<Array [False, True, True, ... False, False] type='28616 * bool'>

In [202]:
perm_counts

<Array [12, 60, 12, 60, 12, ... 60, 12, 12, 12] type='28616 * int64'>

In [206]:
even_perm = np.repeat(even, perm_counts) # [12*False, 60*True, ... ]
even_perm

<Array [False, False, False, ... False, False] type='1596720 * bool'>

In [ ]:
model_even = XGBClassifier()
model_even.load_model('to path')
model_odd = XGBClassifier()
model_odd.load_model('to path')

In [208]:
# def get_inference_results_local(features, even, model_even, model_odd):

results = np.zeros(features.shape[0]) #the total number of permutations, 1596720. Target arrays set to 0

1596720

In [216]:
features[even_perm]

array([[3.53892112e+00, 4.47395992e+00, 1.71641088e+00, ...,
        9.99023438e-01, 1.04248047e-01, 3.93371582e-02],
       [1.69428432e+00, 4.47395992e+00, 1.71641088e+00, ...,
        9.99023438e-01, 1.04248047e-01, 5.28808594e-01],
       [1.00351238e+00, 4.47395992e+00, 2.48113680e+00, ...,
        9.99023438e-01, 3.93371582e-02, 1.04248047e-01],
       ...,
       [3.38696718e+00, 2.91796923e+00, 3.65302038e+00, ...,
        5.94615936e-04, 1.09863281e-01, 9.82910156e-01],
       [1.69450653e+00, 3.65302038e+00, 4.61065245e+00, ...,
        1.09863281e-01, 9.82910156e-01, 5.94615936e-04],
       [3.38696718e+00, 3.65302038e+00, 2.91796923e+00, ...,
        1.09863281e-01, 5.94615936e-04, 9.82910156e-01]])

In [ ]:
if len(features[even_perm])>0: #803448
    results[even_perm] = model_odd.predict_proba(features[even_perm,:])[:,1] # prodict_proba(all features with even_perm mask)(getting all output for class 1=correct classification)
if len(features[np.invert(even_perm)])>0: #793272
    results[np.invert(even_perm)] = model_even.predict_proba(features[np.invert(even_perm), :])[:,1]
# return results

## get_interference_results_triton

## training_filter

In [30]:
def training_filter(jets, electrons, muons, genparts, even):
    '''
    Filters events down to training set and calculates jet-level labels
    
    Args:
        jets: selected jets after region filter (and selecting leading four for each event)
        electrons: selected electrons after region filter
        muons: selected muons after region filter
        genparts: selected genpart after region filter
        even: whether the event is even-numbered (used to separate training events)
    
    Returns:
        jets: selected jets after training filter
        electrons: selected electrons after training filter
        muons: selected muons after training filter
        labels: labels of jets within an event (24=W, 6=top_hadron, -6=top_lepton)
        even: whether the event is even-numbered
    '''
    #### filter genPart to valid matching candidates ####

    # get rid of particles without parents
    genpart_parent = genparts.distinctParent
    genpart_filter = np.invert(ak.is_none(genpart_parent, axis=1))
    genparts = genparts[genpart_filter]
    genpart_parent = genparts.distinctParent

    # ensure that parents are top quark or W
    genpart_filter2 = ((np.abs(genpart_parent.pdgId)==6) | (np.abs(genpart_parent.pdgId)==24))
    genparts = genparts[genpart_filter2]

    # ensure particle itself is a quark
    genpart_filter3 = ((np.abs(genparts.pdgId)<7) & (np.abs(genparts.pdgId)>0))
    genparts = genparts[genpart_filter3]

    # get rid of duplicates
    genpart_filter4 = genparts.hasFlags("isLastCopy")
    genparts = genparts[genpart_filter4]
            
        
    #### get jet-level labels and filter events to training set
        
    # match jets to nearest valid genPart candidate
    nearest_genpart = jets.nearest(genparts, threshold=0.4)
    nearest_parent = nearest_genpart.distinctParent # parent of matched particle
    parent_pdgid = nearest_parent.pdgId # pdgId of parent particle
    grandchild_pdgid = nearest_parent.distinctChildren.distinctChildren.pdgId # pdgId of particle's parent's grandchildren

    grandchildren_flat = np.abs(ak.flatten(grandchild_pdgid,axis=-1)) # flatten innermost axis for convenience

    # if particle has a cousin that is a lepton
    has_lepton_cousin = (ak.sum(((grandchildren_flat%2==0) & (grandchildren_flat>10) & (grandchildren_flat<19)),
                                axis=-1)>0)
    # if particle has a cousin that is a neutrino
    has_neutrino_cousin = (ak.sum(((grandchildren_flat%2==1) & (grandchildren_flat>10) & (grandchildren_flat<19)),
                                  axis=-1)>0)

    # if a particle has a lepton cousin and a neutrino cousin
    has_both_cousins = ak.fill_none((has_lepton_cousin & has_neutrino_cousin), False).to_numpy()

    # get labels from parent pdgId (fill none with 100 to filter out events with those jets)
    labels = np.abs(ak.fill_none(parent_pdgid,100).to_numpy())
    labels[has_both_cousins] = -6 # assign jets with both cousins as top_lepton (not necessarily antiparticle)

    training_event_filter = (np.sum(labels,axis=1)==48) # events with a label sum of 48 have the correct particles
            
    # filter events
    jets = jets[training_event_filter]
    electrons = electrons[training_event_filter]
    muons = muons[training_event_filter]
    labels = labels[training_event_filter]
    even = even[training_event_filter]
    
    return jets, electrons, muons, labels, even


In [31]:
selected_electrons = events.Electron[(events.Electron.pt > 30) & (np.abs(events.Electron.eta)<2.1) & 
                                        (events.Electron.cutBased==4) & (events.Electron.sip3d < 4)]
selected_muons = events.Muon[(events.Muon.pt > 30) & (np.abs(events.Muon.eta)<2.1) & (events.Muon.tightId) & 
                                (events.Muon.sip3d < 4) & (events.Muon.pfRelIso04_all < 0.15)]
jet_filter = (events.Jet.pt > 30) & (np.abs(events.Jet.eta) < 2.4) & (events.Jet.isTightLeptonVeto)
selected_jets = events.Jet[jet_filter]
selected_genpart = events.GenPart
even = (events.event%2==0)
    
# single lepton requirement
event_filters = ((ak.count(selected_electrons.pt, axis=1) + ak.count(selected_muons.pt, axis=1)) == 1)
# require at least 4 jets
event_filters = event_filters & (ak.count(selected_jets.pt, axis=1) >= 4)
# require at least one jet above B_TAG_THRESHOLD
B_TAG_THRESHOLD = 0.5
event_filters = event_filters & (ak.sum(selected_jets.btagCSVV2 >= B_TAG_THRESHOLD, axis=1) >= 1)
    
# apply event filters
selected_electrons = selected_electrons[event_filters]
selected_muons = selected_muons[event_filters]
selected_jets = selected_jets[event_filters]
selected_genpart = selected_genpart[event_filters]
even = even[event_filters]
    
### only consider 4j2b (signal) region
region_filter = ak.sum(selected_jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 2 # at least two b-tagged jets
selected_jets_region = selected_jets[region_filter][:,:4] # only keep top 4 jets
selected_electrons_region = selected_electrons[region_filter]
selected_muons_region = selected_muons[region_filter]
selected_genpart_region = selected_genpart[region_filter]
even = even[region_filter]
    
# filter events and calculate labels
jets, electrons, muons, labels, even = training_filter(selected_jets_region, 
                                                                selected_electrons_region, 
                                                                selected_muons_region, 
                                                                selected_genpart_region,
                                                                even)